In [14]:
!pip install lxml


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip


In [15]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import lxml.html


ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (code signature in <23E270E0-1FC7-3D83-BF2D-3706D65C09E2> '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' not valid for use in process: Trying to load an unsigned library), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (no such file), '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (code signature in <23E270E0-1FC7-3D83-BF2D-3706D65C09E2> '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' not valid for use in process: Trying to 

In [7]:
import os
import re
from datetime import datetime
from typing import Any

import lxml.html
import openpyxl
from lxml import etree
import math


class Fog:
    re_syllables = re.compile(r'(^|[^aeuoiy])(?!e$)[aeouiy]', re.IGNORECASE)

    @classmethod
    def count_syllables(cls, word: str):
        """Counts the number of syllables in a word."""
        syllables_matches = cls.re_syllables.findall(word)
        return len(syllables_matches)

    @classmethod
    def is_complex_word(cls, word: str):
        """Checks whether word has three or more syllables."""
        return cls.count_syllables(word) >= 3

    @classmethod
    def identify_words(cls, input_text: str):
        """Extracts all words from a given text."""
        words = re.findall(r"\b[a-zA-Z\'\-]+\b", input_text)
        return words

    @classmethod
    def count_words(cls, input_text: str):
        """Counts the number of words in a given text."""
        words = cls.identify_words(input_text)
        word_count = len(words)
        return word_count

    @classmethod
    def identify_sentences(cls, input_text: str):
        """Extracts all sentences from a given text."""
        sentences = re.findall(r"\b[A-Z](?:[^\.!?]|\.\d)*[\.!?]", input_text)
        return sentences

    @classmethod
    def count_sentences(cls, input_text: str):
        """Counts the number of sentences in input_text."""
        sentences = cls.identify_sentences(input_text)
        sentence_count = len(sentences)
        return sentence_count

    @classmethod
    def calculate_fog(cls, text: str, *args: Any, **kwds: Any) -> Any:
        """Calculates the fog index for a given text."""
        sentences = cls.identify_sentences(text)
        words = cls.identify_words(text)
        complex_words = list(filter(cls.is_complex_word, words))
        return 0.4*(float(len(words))/float(len(sentences)) + 100*float(len(complex_words))/float(len(words)))


class BaseMDNA:
    @classmethod
    def extract_mdna(cls, html_source: str):
        raise NotImplementedError()

    @classmethod
    def extract_ntfs(cls, html_source: str):
        raise NotImplementedError()

    @classmethod
    def get_text_from_html(cls, html: str):
        """Converts HTML code to plain text"""
        doc = lxml.html.fromstring(html)

        for tag in ["tr", "th", "td", "a", "p", "div", "br", "h1", "h2", "h3", "h4", "h5"]:
            for element in doc.findall(tag):
                if element.text:
                    element.text = element.text + "\n"
                else:
                    element.text = "\n"
        return doc.text_content().strip()

    @classmethod
    def get_mdna_text(cls, html_complete_10k: str, *args: Any, **kwds: Any) -> Any:
        html_mdna_only, year = cls.extract_mdna(html_complete_10k)
        if html_mdna_only is None:
            return None, year

        return cls.get_text_from_html(html_mdna_only), year

    @classmethod
    def get_ntfs_text(cls, html_complete_10k: str, *args: Any, **kwds: Any) -> Any:
        html_ntfs_only, year = cls.extract_ntfs(html_complete_10k)
        if html_ntfs_only is None:
            return None, year

        return cls.get_text_from_html(html_ntfs_only), year


class AppleMDNA(BaseMDNA):
    @classmethod
    def extract_mdna(cls, html_source: str):
        raise NotImplementedError()

    @classmethod
    def extract_ntfs(cls, html_source: str):
        raise NotImplementedError()


def main():
    directory = 'newdata.csv'
    metrics = {}
    profile = openpyxl.Workbook()
    sheet = profile.active
    sheet.append(['Company', 'Year', 'Section', 'Metric', 'Value'])

    for filename in os.listdir(directory):
        if not filename.endswith('.html'):
            continue
        with open(os.path.join(directory, filename), 'r', encoding='windows-1252') as file:
            html_source = file.read()

        mdna_text, year = AppleMDNA.get_mdna_text(html_source)
        ntfs_text, _ = AppleMDNA.get_ntfs_text(html_source)

        if not mdna_text or not ntfs_text:
            continue

        fog_score_mdna = Fog.calculate_fog(mdna_text)
        length_mdna = len(mdna_text)

        fog_score_ntfs = Fog.calculate_fog(ntfs_text)
        length_ntfs = len(ntfs_text)

        metrics.setdefault('APPLE INC', {}).setdefault(year, {})['MD&A'] = (fog_score_mdna, length_mdna)
        metrics['APPLE INC'][year]['NTFS'] = (fog_score_ntfs, length_ntfs)

    for company, company_metrics in metrics.items():
        sorted_years = sorted(company_metrics.keys())
        for i in range(1, len(sorted_years)):
            prev_year, current_year = sorted_years[i - 1], sorted_years[i]
            for section in ['MD&A', 'NTFS']:
                fog_diff = company_metrics[current_year][section][0] - company_metrics[prev_year][section][0]
                length_diff = company_metrics[current_year][section][1] - company_metrics[prev_year][section][1]
                sheet.append([company, current_year, section, 'Fog difference', fog_diff])
                sheet.append([company, current_year, section, 'Length difference', length_diff])

    profile.save('profile.xlsx')


if __name__ == '__main__':
    main()


ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (code signature in <23E270E0-1FC7-3D83-BF2D-3706D65C09E2> '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' not valid for use in process: Trying to load an unsigned library), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (no such file), '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' (code signature in <23E270E0-1FC7-3D83-BF2D-3706D65C09E2> '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lxml/etree.cpython-310-darwin.so' not valid for use in process: Trying to 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Load the data
xl = pd.ExcelFile("data.xlsx")
apple_df = xl.parse("Apple Inc") 

# Calculate the new variables
apple_df['Market_to_Book'] = apple_df['Market Value - Total'] / apple_df['Stockholders Equity - Total']
apple_df['Earnings'] = apple_df['Common Shares Used to Calculate Earnings Per Share - Basic'] * apple_df['Earnings Per Share (Basic) - Including Extraordinary Items']

# Calculate the differences
apple_df['Fog_diff'] = apple_df['Fog'].diff()
apple_df['Length_diff'] = apple_df['Length'].diff()



# Calculate the summary statistics
summary_statistics = apple_df[['Year', 'Earnings', 'Market_to_Book', 'Market Value - Total', 'Assets - Total', 'Fog', 'Fog_diff', 'Length', 'Length_diff']].describe()
print(summary_statistics)

# Calculate the correlation matrix
correlation_matrix = apple_df[['Year', 'Earnings', 'Market_to_Book', 'Market Value - Total', 'Assets - Total', 'Fog', 'Fog_diff', 'Length', 'Length_diff']].corr()
print(correlation_matrix)

# Add year t-1 for Fog and Length
apple_df['Fog_lag1'] = apple_df['Fog'].shift(1)
apple_df['Length_lag1'] = apple_df['Length'].shift(1)

# Add quintiles for Fog and Length at t and t-1
apple_df['Fog_quintile'] = pd.qcut(apple_df['Fog'], 5, labels=False)
apple_df['Fog_lag1_quintile'] = pd.qcut(apple_df['Fog_lag1'], 5, labels=False)

apple_df['Length_quintile'] = pd.qcut(apple_df['Length'], 5, labels=False)
apple_df['Length_lag1_quintile'] = pd.qcut(apple_df['Length_lag1'], 5, labels=False)

# Filter the first and fifth quintiles
first_quintile = apple_df[(apple_df['Fog_lag1_quintile'] == 0) | (apple_df['Length_lag1_quintile'] == 0)]
fifth_quintile = apple_df[(apple_df['Fog_lag1_quintile'] == 4) | (apple_df['Length_lag1_quintile'] == 4)]

# Calculate transition matrix for Fog and Length
fog_transition_matrix_1st = pd.crosstab(first_quintile['Fog_lag1_quintile'], first_quintile['Fog_quintile'], normalize='index')
fog_transition_matrix_5th = pd.crosstab(fifth_quintile['Fog_lag1_quintile'], fifth_quintile['Fog_quintile'], normalize='index')

length_transition_matrix_1st = pd.crosstab(first_quintile['Length_lag1_quintile'], first_quintile['Length_quintile'], normalize='index')
length_transition_matrix_5th = pd.crosstab(fifth_quintile['Length_lag1_quintile'], fifth_quintile['Length_quintile'], normalize='index')

print("Fog Transition Matrix for 1st Quintile:")
print(fog_transition_matrix_1st)

print("Fog Transition Matrix for 5th Quintile:")
print(fog_transition_matrix_5th)

print("Length Transition Matrix for 1st Quintile:")
print(length_transition_matrix_1st)

print("Length Transition Matrix for 5th Quintile:")
print(length_transition_matrix_5th)


In [2]:
import pandas as pd
data = pd.read_csv("COMPUSTATEARNINGSCRSPRETURNSMERGED.csv")
data

,Standard and Poor's Identifier,Primary Link Marker,Security-level Identifier,Link Type Codd,Historical CRSP PERMNO Link to COMPUSTAT Record,Historical CRSP PERMCO Link to COMPUSTAT Record,First Effective Date of Link,Last Effective Date of Link,Data Date,Fiscal Year,...,Current Primary Issue Tag - US,Standard Industry Classification Code,S&P Industry Sector Code,S&P Economic Sector Code,S&P Quality Ranking - Current,State/Province,Stock Ownership Code,Web URL,Research Company Deletion Date,Company Initial Public Offering Date
0,1690,P,1,LU,14593,7,12/12/80,NaN,6/30/11,2011,...,1,3663,190.0,940.0,A,CA,0,www.apple.com,NaN,12/12/80
1,1690,P,1,LU,14593,7,12/12/80,NaN,9/30/11,2011,...,1,3663,190.0,940.0,A,CA,0,www.apple.com,NaN,12/12/80
2,1690,P,1,LU,14593,7,12/12/80,NaN,12/31/11,2012,...,1,3663,190.0,940.0,A,CA,0,www.apple.com,NaN,12/12/80
3,1690,P,1,LU,14593,7,12/12/80,NaN,3/31/12,2012,...,1,3663,190.0,940.0,A,CA,0,www.apple.com,NaN,12/12/80
4,1690,P,1,LU,14593,7,12/12/80,NaN,6/30/12,2012,...,1,3663,190.0,940.0,A,CA,0,www.apple.com,NaN,12/12/80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,170617,P,1,LC,13407,54084,5/18/12,NaN,9/30/21,2021,...,1,7370,NaN,NaN,NaN,CA,0,investor.fb.com,NaN,5/18/12
256,170617,P,1,LC,13407,54084,5/18/12,NaN,12/31/21,2021,...,1,7370,NaN,NaN,NaN,CA,0,investor.fb.com,NaN,5/18/12
257,170617,P,1,LC,13407,54084,5/18/12,NaN,3/31/22,2022,...,1,7370,NaN,NaN,NaN,CA,0,investor.fb.com,NaN,5/18/12
258,170617,P,1,LC,13407,54084,5/18/12,NaN,6/30/22,2022,...,1,7370,NaN,NaN,NaN,CA,0,investor.fb.com,NaN,5/18/12


In [3]:
import pandas as pd

# read 'result.xlsx' file
result_data = pd.read_excel('result.xlsx')

# replace company names with ticker symbols
result_data['company'] = result_data['company'].replace({
    'APPLE INC': 'AAPL',
    'NETFLIX INC': 'NFLX',
    'FACEBOOK INC': 'META',
    'AMAZON COM INC': 'AMZN',
    'GOOGLE INC': 'GOOGL'
})

# rename 'company' column to 'Ticker Symbol'
result_data = result_data.rename(columns={'company': 'Ticker Symbol'})


In [4]:
data = pd.merge(data, result_data[['MD&A','year','Ticker Symbol', 'fog']], on='Ticker Symbol')

# Previous Parts

In [12]:
# Filter the data for the five firms between 2012 and 2021
data = data[(data['Ticker Symbol'].isin(['AAPL', 'AMZN', 'NFLX', 'GOOGL', 'META'])) & 
            (data['Fiscal Year'].between(2012, 2021))]

# Calculate the variables needed for the regression
data['size'] = data['Assets - Total'].apply(lambda x: 0 if x <= 0 else pd.np.log(x))
data['market_to_book'] = data['Market Value - Total'] / data['Stockholders Equity > Parent > Index Fundamental > Quarterly']
data['special_items'] = data['Operating Income Before Depreciation - Quarterly'] - data['Net Income (Loss)']
data['complexity_of_operations'] = data['Cost of Goods Sold'] / data['Sales/Turnover (Net)']
data['incorporation_state'] = data['North American Industry Classification Code']  # Assuming NAICS represents the incorporation state, as it is not specified in the question

data = data[['fog', 'size', 'market_to_book', 'special_items', 'complexity_of_operations', 'incorporation_state']]

# Filter the columns needed for the regression
regression_data = data[['fog', 'size', 'market_to_book', 'special_items', 'complexity_of_operations', 'incorporation_state']]


/var/folders/m6/wxc9fp790875bb7z5jw_q5c80000gn/T/ipykernel_14094/756979738.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  data['size'] = data['Assets - Total'].apply(lambda x: 0 if x <= 0 else pd.np.log(x))


In [13]:
# Define the regression formula
formula = 'fog ~ size + market_to_book + special_items + complexity_of_operations + C(incorporation_state)'

# Perform the regression
model = smf.ols(formula, data=regression_data).fit()

# Print the results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    fog   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     456.5
Date:                Fri, 26 May 2023   Prob (F-statistic):               0.00
Time:                        14:47:52   Log-Likelihood:                -1887.2
No. Observations:                2152   AIC:                             3792.
Df Residuals:                    2143   BIC:                             3844.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep